In [1]:
import os

import tensorflow as tf
import tensorflow_transform as tft
import tfx
from tfx.components import (ImportExampleGen, StatisticsGen, SchemaGen,
                            ExampleValidator, Transform, Trainer)
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

2023-01-13 09:20:38.008558: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Model Training
전처리 단계에서 생성한 전처리 그래프를 모델의 일부로 병합시켜야 한다. 모델이 요구하는 피처 표현을 항상
보장할 수 있기 때문이다. 이때 전처리 컴포넌트를 통과한 상태의 피처의 이름은 훈련 모델의 입력 노드의 이름과
동일해야 한다.

TFX에 호환되는 모델 종류는 Keras, Estimator 두 가지인데 Estimator는 [공식 문서](https://www.tensorflow.org/api_docs/python/tf/estimator/Estimator)에서 권장하지
않는 훈련 방식이다.

## `run_fn()` 정의
`Transform` 컴포넌트가 `preprocessing_fn`을 포함한 모듈을 요구하듯, `Trainer` 컴포넌트도
`run_fn`이 정의된 모듈이 필요하다. 이 함수는 다음 역할을 하며 이를 함수 내부에 기술해야 한다.
1. 입력 데이터(train, eval 데이터셋 혹은 입력 함수도 가능 - 함수형!) 준비
2. 모델 빌드, 컴파일
3. 모델 학습
4. 학습된 모델 내보내기

`Trainer` 컴포넌트는 작동되기 시작하면(대화형에선 `context.run(trainer)`) `run_fn`이 포함된
모듈부터 찾아 진입점으로 삼는다.

In [ ]:
# `run_fn` 함수 작성을 위해 필요한 변수 및 헬퍼 함수들.

# 피처명, 차원(가능한 값의 수)
ONE_HOT_FEATURES = {
    "product": 11,
    "sub_product": 45,
    "company_response": 5,
    "state": 60,
    "issue": 90,
}

# 피처명, 버킷 수(범위 수)
BUCKET_FEATURES = {
    "zip_code": 10
}

# 피처명, 미정의
TEXT_FEATURES = {
    "consumer_complaint_narrative": None
}

LABEL_KEY = "consumer_disputed"


def transform_name(key):
    return key + "_xf"


def _gzip_reader_fn(filenames):
    return tf.data.TFRecordDataset(filenames, compression_type="GZIP")


def input_fn(file_pattern, tf_transform_output, batch_size=32):
    # 미리 변환된 데이터셋을 불러오도록 하려면 피처를 기술한 feature spec이 필요하다.
    # 대신 그 경우 변환과 추론 그래프가 통합된 모델 내보내기란 장점을 포기해야 한다.
    transformed_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    # `make_batched_features_dataset`는 배치별 데이터 제공 제너레이터를 반환한다.
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transformed_feature_spec,
        reader=_gzip_reader_fn,
        label_key=transform_name(LABEL_KEY)
    )

    return dataset

In [ ]:
def get_model():
    import tensorflow_hub as hub

    input_features = []
    # 피처를 루프 돌리며 각 피처에 대한 input_feature를 작성합니다.
    for key, dim in ONE_HOT_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,),
                           name=transform_name(key)))

    # 버킷화 피처를 추가
    for key, dim in BUCKET_FEATURES.items():
        input_features.append(
            tf.keras.Input(shape=(dim + 1,),
                           name=transform_name(key)))

    # 문자열 피처를 추가
    input_texts = []
    for key in TEXT_FEATURES.keys():
        input_texts.append(
            tf.keras.Input(shape=(1,),
                           name=transform_name(key),
                           dtype=tf.string))

    inputs = input_features + input_texts

    # 문자열 피처를 임베딩
    MODULE_URL = "https://tfhub.dev/google/universal-sentence-encoder/4"
    # 범용 문장 인코더 모델의 tf.hub 모듈을 로드합니다.
    embed = hub.KerasLayer(MODULE_URL)
    # 케라스 입력은 2차원이지만 인코더는 1차원 입력을 기대합니다.
    reshaped_narrative = tf.reshape(input_texts[0], [-1])
    embed_narrative = embed(reshaped_narrative)
    deep_ff = tf.keras.layers.Reshape((512,), input_shape=(1, 512))(embed_narrative)

    deep = tf.keras.layers.Dense(256, activation='relu')(deep_ff)
    deep = tf.keras.layers.Dense(64, activation='relu')(deep)
    deep = tf.keras.layers.Dense(16, activation='relu')(deep)

    wide_ff = tf.keras.layers.concatenate(input_features)
    wide = tf.keras.layers.Dense(16, activation='relu')(wide_ff)

    both = tf.keras.layers.concatenate([deep, wide])

    output = tf.keras.layers.Dense(1, activation='sigmoid')(both)
    # 피처 API로 모델 그래프를 조립합니다.
    keras_model = tf.keras.models.Model(inputs, output)

    keras_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        loss='binary_crossentropy',
                        metrics=[
                            tf.keras.metrics.BinaryAccuracy(),
                            tf.keras.metrics.TruePositives()
                        ])
    return keras_model

In [ ]:
# 이 부분을 출력 시그니처(`serving_default`)에 매핑함으로써 변환 단계를 모델에 병합하게 된다.
def _get_serve_tf_examples_fn(model, tf_transform_output):
    # 변환 그래프 로드.
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        # 변환 전 직렬화된 원시 데이터를 피처 사양으로 구조화.
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        # 원시 데이터에 변환 적용.
        transformed_features = model.tft_layer(parsed_features)
        # 변환된 데이터로 예측.
        outputs = model(transformed_features)

        return {"outputs": outputs}

    return serve_tf_examples_fn


def run_fn(fn_args):
    # 1. 입력 데이터 준비
    # `fn_args.transform_output`: `Transform`을 거쳐 나온 변환 그래프, 예제 데이터셋,
    # 학습 매개변수 등
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)
    train_dataset = input_fn(fn_args.train_files, tf_transform_output)
    eval_dataset = input_fn(fn_args.eval_files, tf_transform_output)

    # 2. 모델 빌드 및 컴파일
    model = get_model()

    # 3. 모델 훈련
    # epochs==n으로 학습하려면 steps=(num_batches * n)으로 설정하면 된다.
    model.fit(
        train_dataset,
        steps_per_epoch=fn_args.train_steps,
        validation_data=eval_dataset,
        validation_steps=fn_args.eval_steps
    )

    # 4. 학습된 모델 내보내기
    signatures = {
        "serving_default": _get_serve_tf_examples_fn(
            model, tf_transform_output).get_concrete_function(
            tf.TensorSpec(shape=[None], dtype=tf.string, name="examples")
        )
    }
    model.save(fn_args.serving_model_dir, save_format="tf", signatures=signatures)

## `Trainer` 실행

In [3]:
context = InteractiveContext()

In [4]:
# 1. ExampleGen
data_dir = os.path.join(os.getcwd(), "data")
record_dir = os.path.join(data_dir, "complaints", "records")
example_gen = ImportExampleGen(input_base=record_dir)
context.run(example_gen)

ExecutionResult(
    component_id: ImportExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=ImportExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}))

In [5]:
examples = example_gen.outputs["examples"]

In [6]:
# 2. StatisticsGen: 피처 요약 통계 생성.
statistics_gen = StatisticsGen(examples=examples)
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}))

In [7]:
stats = statistics_gen.outputs["statistics"]

In [8]:
# 3. SchemaGen: 피처 통계로부터 스키마 추론.
schema_gen = SchemaGen(statistics=stats, infer_feature_shape=True)
context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}))

In [9]:
schema = schema_gen.outputs["schema"]

In [10]:
example_validator = ExampleValidator(statistics=stats, schema=schema)
context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}))

In [11]:
transform = Transform(examples=examples,
                      schema=schema,
                      module_file="transform_module.py")
context.run(transform)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform_module.py -> build/lib
copying trainer_module.py -> build/lib
copying hello.py -> build/lib
installing to /tmp/tmp6k9cn_w_
running install
running install_lib
copying build/lib/transform_module.py -> /tmp/tmp6k9cn_w_
copying build/lib/trainer_module.py -> /tmp/tmp6k9cn_w_
copying build/lib/hello.py -> /tmp/tmp6k9cn_w_
running install_egg_info
running egg_info
creating tfx_user_code_Transform.egg-info
writing tfx_user_code_Transform.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Transform.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Transform.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Transform.egg-info/SOURCES.txt'
Copying tfx_user_code_Transform.egg-info to /tmp/tmp6k9cn_w_/tfx_user_code_Trans

/home/hakjun/projects/pipeline/venv/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


adding 'transform_module.py'
adding 'tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f.dist-info/METADATA'
adding 'tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f.dist-info/WHEEL'
adding 'tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f.dist-info/top_level.txt'
adding 'tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f.dist-info/RECORD'
removing /tmp/tmp6k9cn_w_
Processing /tmp/tfx-interactive-2023-01-13T09_23_05.899934-2hbg5n4m/_wheels/tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f-py3-none-any.whl
Processing /tmp/tfx-interactive-2023-01-13T09_23_05.899934-2hbg5n4m/_wheels/tfx_user_code_Transform-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f-py3-none-any.whl
Processing /tmp/tfx-interactive-2023-01-13T09_23_05.899934-2hbg5n4m/_wheels/tfx_user_code

2023-01-13 09:23:50.945977: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Use ref() instead.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={})
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={})
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={})
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={})
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={})
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={})
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}))

In [12]:
transform_graph = transform.outputs["transform_graph"]
transformed_examples = transform.outputs["transformed_examples"]

In [13]:
trainer = Trainer(
    module_file="trainer_module.py",
    examples=transformed_examples,
    transform_graph=transform_graph,
    schema=schema,
    train_args=tfx.proto.trainer_pb2.TrainArgs(num_steps=500),  # prev: 100
    eval_args=tfx.proto.trainer_pb2.EvalArgs(num_steps=100))  # prev: 50
# 실행이 끝난 후 출력 아티팩트 메타데이터(생성된 모델 경로 등)은 MLMD에 저장된다.
# context의 경우엔 암시적으로 메모리 내에 메타데이터스토어가 구축된 상태임을 상기하자.
context.run(trainer)

running bdist_wheel
running build
running build_py
creating build
creating build/lib
copying transform_module.py -> build/lib
copying trainer_module.py -> build/lib
copying hello.py -> build/lib
installing to /tmp/tmpod4ff78a
running install
running install_lib
copying build/lib/transform_module.py -> /tmp/tmpod4ff78a
copying build/lib/trainer_module.py -> /tmp/tmpod4ff78a
copying build/lib/hello.py -> /tmp/tmpod4ff78a
running install_egg_info
running egg_info
creating tfx_user_code_Trainer.egg-info
writing tfx_user_code_Trainer.egg-info/PKG-INFO
writing dependency_links to tfx_user_code_Trainer.egg-info/dependency_links.txt
writing top-level names to tfx_user_code_Trainer.egg-info/top_level.txt
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
reading manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
writing manifest file 'tfx_user_code_Trainer.egg-info/SOURCES.txt'
Copying tfx_user_code_Trainer.egg-info to /tmp/tmpod4ff78a/tfx_user_code_Trainer-0.0+3100132ed

/home/hakjun/projects/pipeline/venv/lib/python3.8/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(


Processing /tmp/tfx-interactive-2023-01-13T09_23_05.899934-2hbg5n4m/_wheels/tfx_user_code_Trainer-0.0+3100132ed8ea60cb8773f45a92a4da856ccdce929d65b49a4bf28e3401c1242f-py3-none-any.whl


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2023-01-13T09_23_05.899934-2hbg5n4m/Trainer/model/6/Format-Serving/assets


INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2023-01-13T09_23_05.899934-2hbg5n4m/Trainer/model/6/Format-Serving/assets


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={})
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}))

In [14]:
model_artifact = trainer.outputs["model"]
model_uri = model_artifact.get()[0].uri
model_temp_dir = os.path.join(model_uri, "Format-Serving")

In [15]:
print(model_temp_dir)
os.listdir(model_temp_dir)

/tmp/tfx-interactive-2023-01-13T09_23_05.899934-2hbg5n4m/Trainer/model/6/Format-Serving


['saved_model.pb',
 'fingerprint.pb',
 'assets',
 'keras_metadata.pb',
 'variables']

In [15]:
# Tensorboard 학습 진행 경과 확인(노트북 밖에선 `$ tensorboard` 명령어로 확인 가능. 외부 서버에 열고 접속하려면 `--bind_all` 옵션 필요).
%load_ext tensorboard
%tensorboard --logdir {model_uri}

Launching TensorBoard...

In [ ]:
trained_model = tf.saved_model.load(export_dir=model_temp_dir)
predict_fn = trained_model.signatures["serving_default"]

`predict_fn`은 concrete_function으로서 tf.Example을 입력으로 받는 함수이다(전처리 과정이
포함돼 있으니 Example은 원시 데이터를 그대로 사용해서 만들면 된다).
```Python
example = tf.train.Example(features=tf.train.Features(
    feature={
        "feat1": _bytes_list_feature(val),
    }
))
```

## 하이퍼파라미터 튜닝
`tfx.components.Tuner` 출력 아티팩트를 `Trainer.hyperparameters`의 실인자로 넘기는 방식으로
튜닝이 가능. `Triner`에게 `run_fn`혹은 이 함수가 담긴 모듈을 넘기는 것처럼 `Tuner`에는
`tuner_fn` 혹은 이 함수를 담은 모듈을 전달해야 한다.